In [84]:
#Khai báo thư viện
import cvxpy as cp
import numpy as np
import folium
from scipy.spatial.distance import cdist


In [85]:
# Load the data
# file POI QUẬN 1.csv chứa thông tin về các tiềm năng có thể đặt trụ sạc
# file DÂN CƯ QUẬN 1 - Quận 1.csv chứa thông tin về các điểm dân cư, được tính theo phường và đại diện bằng UBND phường
poi_data = pd.read_csv('POI QUẬN 1.csv')
demand_node_data = pd.read_csv('DÂN CƯ QUẬN 1 - Quận 1.csv')
# Clean the demand node data
demand_node_data['Latitude'] = demand_node_data['Latitude'].str.replace(',', '.').astype(float)
demand_node_data['Longitude'] = demand_node_data['Longitude'].str.replace(',', '.').astype(float)
demand_node_data['Land cost'] = demand_node_data['Land cost'].str.replace('.', '').str.replace(',', '.').astype(float)

In [87]:
# Set installation and land costs
poi_coords = poi_data[['Latitude', 'Longitude']].dropna().values
n = len(poi_coords)
i2i = np.array([50] * n)  # Installation cost for level 2
i3i = np.array([100] * n)  # Installation cost for level 3
li = np.array([100] * n)  # Land rental cost
r = 365  # Average days needed to install a station
V2 = 10  # Average charging speed level 2
V3 = 20  # Average charging speed level 3
dj = np.array([1] * n)  # Charging demand in area j (example values)
m2 = 100  # Maintenance cost for level 2
m3 = 150  # Maintenance cost for level 3
e2 = 5  # Electricity cost level 2
e3 = 7  # Electricity cost level 3
Po = 50  # Average daily electricity cost borne by station owner
Pu = 75  # Average daily electricity revenue from customers
eu2 = 5  # Electricity cost for customers level 2
eu3 = 7  # Electricity cost for customers level 3
w = 10000  # Average income per capita in HCMC
q2 = 1  # Waiting time for level 2
q3 = 1.5  # Waiting time for level 3
theta_ij = np.array([10] * len(demand_node_data))  # Distance from residential area (example values)
C = 10  # Electricity cost per km
# Decision variables for level 2 and level 3 stations
x2 = cp.Variable(n, integer=True)
x3 = cp.Variable(n, integer=True)
# Calculate the distance between each demand node and each POI
demand_coords = demand_node_data[['Latitude', 'Longitude']].values
distance_matrix = cdist(demand_coords, poi_coords, metric='euclidean')

In [89]:
# Define the objective function (cost to minimize)
owner_cost = cp.sum(cp.multiply(x2, i2i) + cp.multiply(x3, i3i)) + \
             cp.sum(cp.multiply(x2, 24**2 * li * r * V2) + cp.multiply(x3, li * r * V3)) / (365 * cp.sum(dj)) + \
             cp.sum(cp.multiply(x2, m2) + cp.multiply(x3, m3) + Po * (cp.multiply(x2, e2) + cp.multiply(x3, e3)))

user_cost = cp.sum(cp.multiply(Pu, (cp.multiply(x2, eu2) + cp.multiply(x3, eu3)))) + \
            cp.sum(cp.multiply(w, (x2 * q2 + x3 * q3))) + \
            cp.sum(cp.multiply(theta_ij, C))

total_cost = owner_cost + user_cost

In [90]:
# Constraints
constraints = [
    x2 >= 0,  # Non-negative number of level 2 stations
    x3 >= 0,  # Non-negative number of level 3 stations
    x2 + x3 <= 1,  # Only one station (level 2 or 3) per POI
    cp.sum(x2 + x3) == 10  # Exactly 10 stations should be placed
]

In [91]:
# Define the problem
problem = cp.Problem(cp.Minimize(total_cost), constraints)

# Solve the problem
problem.solve()

# Extract the optimal placement of charging stations
optimal_x2 = x2.value.round().astype(int)
optimal_x3 = x3.value.round().astype(int)

# Combine results into a single array to identify selected POIs
optimal_placement = optimal_x2 + optimal_x3
optimal_pois = poi_data.iloc[optimal_placement > 0]

# Create a folium map centered around central Ho Chi Minh City
m = folium.Map(location=[10.77584, 106.70098], zoom_start=14)

# Add demand nodes (user homes) to the map
for _, row in demand_node_data.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=row['Địa điểm'],
        icon=folium.Icon(color='red', icon='home')
    ).add_to(m)

# Add selected POIs (charging stations) to the map
for _, row in optimal_pois.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=row['Ten DN/Địa điểm'],
        icon=folium.Icon(color='blue', icon='plug')
    ).add_to(m)

# Save and display the map
m.save('charging_stations_map.html')
m
